**Brian Blaylock**  
**26 February 2019**

# Compute the area of a closed lat/lon path
Source: https://stackoverflow.com/questions/4681737/how-to-calculate-the-area-of-a-polygon-on-the-earths-surface-using-python


In [11]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches


import sys
sys.path.append('/uufs/chpc.utah.edu/common/home/u0553130/pyBKB_v3')
from BB_HRRR.HRRR_Pando import get_hrrr_variable
from BB_cmap.landuse_colormap import LU_MODIS21
from BB_cmap.FLAME_cmap import cm_ros
from BB_maps.my_basemap import draw_HRRR_map, draw_centermap

In [9]:
H = get_hrrr_variable(datetime(2019,1,1), 'HGT:surface')
H_land = get_hrrr_variable(datetime(2019,1,1), 'LAND:surface')


 >> Dowloading tempfile: ./temp_hrrr_2019010100_f00_HGT.grib2
Oh, good, you requested a date that should be on Pando.
GRIB2 File: https://pando-rgw01.chpc.utah.edu/hrrr/sfc/20190101/hrrr.t00z.wrfsfcf00.grib2
 .idx File: https://pando-rgw01.chpc.utah.edu/hrrr/sfc/20190101/hrrr.t00z.wrfsfcf00.grib2.idx
 >> Matched a variable:  58:30896245:d=2019010100:HGT:surface:anl:
 >> Byte Range: 30896245 33131021
  Run Date: 2019-01-01 00:00 UTC F00
Valid Date: 2019-01-01 00:00 UTC

 >> Dowloading tempfile: ./temp_hrrr_2019010100_f00_LAN.grib2
Oh, good, you requested a date that should be on Pando.
GRIB2 File: https://pando-rgw01.chpc.utah.edu/hrrr/sfc/20190101/hrrr.t00z.wrfsfcf00.grib2
 .idx File: https://pando-rgw01.chpc.utah.edu/hrrr/sfc/20190101/hrrr.t00z.wrfsfcf00.grib2.idx
 >> Matched a variable:  143:101738055:d=2019010100:LAND:surface:anl:
 >> Byte Range: 101738055 101788530
  Run Date: 2019-01-01 00:00 UTC F00
Valid Date: 2019-01-01 00:00 UTC


In [12]:
PATH_points = {
    'HRRR':
        {'lon': np.concatenate([H['lon'][0], H['lon'][:,-1], H['lon'][-1][::-1], H['lon'][:,0][::-1]]),
         'lat': np.concatenate([H['lat'][0], H['lat'][:,-1], H['lat'][-1][::-1], H['lat'][:,0][::-1]])},
    'West':{
        'lon':[-120, -104, -104, -120, -120],
        'lat':[24.4, 24.4, 50.2, 50.2, 24.2]},
    'Central':{
        'lon':[-104, -88, -88, -104, -104],
        'lat':[24.4, 24.4, 50.2, 50.2, 24.2]},
    'East':{
        'lon':[-88, -72, -72, -88, -88],
        'lat':[24.4, 24.4, 50.2, 50.2, 24.2]},
    'Utah':{
        'lon':[-114.041664, -111.047526, -111.045645,  -109.051460, -109.048632, -114.051534, -114.041664],
        'lat':[41.993580, 42.002846, 40.998538, 40.998403, 36.998310, 37.000574, 41.993580]}
}

## Combine lat/lon as vertice pair as a tuple. i.e. (lon, lat).
PATH_verts = {}
for i in PATH_points.keys():
    PATH_verts[i] = np.array([(PATH_points[i]['lon'][j], PATH_points[i]['lat'][j]) for j in range(len(PATH_points[i]['lon']))])

## Generate Path objects from the vertices.
PATHS = {}
for i in PATH_verts.keys():
    PATHS[i] = Path(PATH_verts[i])

In [13]:
PATHS

{'HRRR': Path(array([[-122.719528  ,   21.138123  ],
        [-122.69286132,   21.14511004],
        [-122.6661903 ,   21.1520901 ],
        ...,
        [-122.73452632,   21.18786863],
        [-122.72702499,   21.16299459],
        [-122.719528  ,   21.138123  ]]), None),
 'West': Path(array([[-120. ,   24.4],
        [-104. ,   24.4],
        [-104. ,   50.2],
        [-120. ,   50.2],
        [-120. ,   24.2]]), None),
 'Central': Path(array([[-104. ,   24.4],
        [ -88. ,   24.4],
        [ -88. ,   50.2],
        [-104. ,   50.2],
        [-104. ,   24.2]]), None),
 'East': Path(array([[-88. ,  24.4],
        [-72. ,  24.4],
        [-72. ,  50.2],
        [-88. ,  50.2],
        [-88. ,  24.2]]), None),
 'Utah': Path(array([[-114.041664,   41.99358 ],
        [-111.047526,   42.002846],
        [-111.045645,   40.998538],
        [-109.05146 ,   40.998403],
        [-109.048632,   36.99831 ],
        [-114.051534,   37.000574],
        [-114.041664,   41.99358 ]]), None)}

In [17]:
PATHS['Utah'].vertices

array([[-114.041664,   41.99358 ],
       [-111.047526,   42.002846],
       [-111.045645,   40.998538],
       [-109.05146 ,   40.998403],
       [-109.048632,   36.99831 ],
       [-114.051534,   37.000574],
       [-114.041664,   41.99358 ]])

In [18]:


#coordinates=numpy.array([
#[-102.05, 41.0], 
#[-102.05, 37.0], 
#[-109.05, 37.0], 
#[-109.05, 41.0],
#[-102.05, 41.0]])

coordinates = PATHS["Utah"].vertices


lats=coordinates[:,1]
lons=coordinates[:,0]

lat1=numpy.min(lats)
lat2=numpy.max(lats)
lon1=numpy.min(lons)
lon2=numpy.max(lons)

# Cylindrial Equal-Area Projection
bmap=Basemap(projection='cea',llcrnrlat=lat1,llcrnrlon=lon1,urcrnrlat=lat2,urcrnrlon=lon2)
xs,ys=bmap(lons,lats)

area=np.abs(0.5*np.sum(ys[:-1]*np.diff(xs)-xs[:-1]*np.diff(ys)))

area=area/1e6

print(area)

219736.18447189653


In [23]:
np.diff([1,4,8,16])

array([3, 4, 8])